## konlpy

In [1]:
from konlpy.tag import Okt
# streamlit error 
# sol) update tweepy to 3.10.0

In [2]:
okt = Okt()

# systemerror ( already loaded in another classloader
# sol) In env\konlpy\jvm.py, every * to be deleted from folder_suffix
# https://byeon-sg.tistory.com/entry/자연어-처리-konlpy-설치-오류-okt에러-already-loaded-in-another-classloader-SystemErro-1 [wave])

In [3]:
# Pos-tagging(형태소 분석)
text = '나는 코딩을 무지무지 좋아합니다!'
okt.pos(text)

[('나', 'Noun'),
 ('는', 'Josa'),
 ('코딩', 'Noun'),
 ('을', 'Josa'),
 ('무지무지', 'Adverb'),
 ('좋아합니다', 'Adjective'),
 ('!', 'Punctuation')]

In [89]:
# Pos-tagging(형태소 분석)
text = '나는 코딩을 무지무지 좋아합니다!'

#pos와는 다르게 단순 나누기만함
okt.morphs(text)

['나', '는', '코딩', '을', '무지무지', '좋아합니다', '!']

In [90]:
# Extracting Nouns

text = '아이폰이랑 갤럭시중 어느 게 더 좋은 핸드폰이야??'
okt.nouns(text)

# 간혹 명사 이외의 것(게, 더)가 명사로 인식된다

['아이폰', '갤럭시', '게', '더', '핸드폰']

In [91]:
# 정규화 : 어지럽힌 문장을 깔끔하게 처리
text = '안녕하세욬ㅋㅋㅋ 반가워요 샤릉해'
print(okt.normalize(text))


안녕하세요ㅋㅋㅋ 반가워요 사랑해


## BOW 실습

In [137]:
# bag of words
# 간단하게 영어 단어 제거 => 정규화 및 필요 없는 단어, 특수문자 제거 => bow

from konlpy.tag import Okt
import re
import pandas as pd

okt = Okt()
df = pd.read_excel('results.xlsx')
df.info()
df = df.fillna(' ')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47 entries, 0 to 46
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   comment    47 non-null     object
 1   author     47 non-null     object
 2   date       47 non-null     object
 3   num_likes  47 non-null     int64 
dtypes: int64(1), object(3)
memory usage: 1.6+ KB


In [139]:
print(df['comment'])

0                                             진짜 기여워잉ㅠㅠ
1                                망고처럼 꽃돌이가 너무 귀엽네옇ㅎㅎㅎㅎㅎ
2                                           귀여운 새침이 ❤❤❤
3                                        꽃돌이 남자에요? 여자에오
4                                           꽃돌이는 여자에요~😄
5                                              아 너무 귀여워
6           제가 키웠던 토끼가 자기 집 위에 올라가서 못 내려오던 이유가 있었군요...ㅋ
7                                이쁜것 ㅠㅠ아휴 ! 꽃돌이 관절 소즁해!
8     이꽃돌씨와 우리 깜토(까미토끼)가만나면 어덯게 될까요?ㅋㅋ<br>큰토끼님!답글 부탁...
9     @꽃돌이는배고파 FlowerisHungry 헉!감사해요!ㅠㅠ나중에 한번 될수있으면 ...
10                        깜토랑 만나면 좋은 친구가 될 수 있을 것 같아요~😍
11                                  꽃돌  왜 이렇게 귀여워서 난리양💕
12    토끼한테 뽀뽀해주면 싫다고 앞발로 미는데 다리가 짧아서 집사는 같이 뽀뽀해주는 걸로...
13       꽃돌이도 이불을 씹는구나 ㅜㅜ 톡톡톡하면서 망가지는 이불들이 남아 나질 않아..ㅜㅜ
14                덕분에 매 시즌 새로운 이불을 살 수 있다는 장점(?)이 있습니다!
15                                     꽃돌이 몸무게 몇kg 나가요?
16               @꽃돌이는배고파 FlowerisHungry 꽃돌이가 살이많이쪘군요^^
17                                          2.5k

In [135]:
# 1. 필요 없는(이모티콘 등 의미 없는) 부분 전처리
# 2. youtube comments에 @nickname~ 이런 경우가 있어, author column의 문자열들 모두 comments에서 모두 삭제해야 한다.

# https://stackoverflow.com/questions/10968558/python-re-sub-with-a-list-of-words-to-find


# 해결 : authors_regex에서 사전으로 특수문자 제거(메타 char 될 수 있는거) 제거 후, authors_regex부터 제거

author = set()
authors = set(['@' + re.sub('\W+', ' ', i) for i in df['author']])
authors_regex = "|".join(authors)

for line in df['comment']:
    line = re.sub('\W+',' ', line)           # 특수 문자 스페이스로 대체.
    line = re.sub(authors_regex, ' ', line)     # 특별한 의미 없는 단어 스페이스로 대체.

comments = []
for line in df['comment']:

        line = re.sub(authors_regex, ' ', line)  # 댓글 중 tag한 거 다 제거
        line = re.sub('\W+',' ', line)           # 특수 문자 스페이스로 대체.
        line = re.sub('\d+',' ', line)         # 수치 스페이스로 대체.
        line = re.sub('\n',' ',line)             # line return 스페이스로 대체.
        line = re.sub('[\[\]]', ' ',line)        # 대괄호 스페이스로 대체.
        line = re.sub('[a-zA-Z]',' ',line)       # 영문 스페이스로 대체.
        line = re.sub('[ㄱ-ㅎ | ㅏ-ㅣ]', ' ', line)     # 단모음, 단자음 스페이스로 대체.
        line = re.sub('\s+', ' ', line)          # 잉여 스페이즈 줄임.
        # line = okt.normalize(line)            # 시간이 너무 걸림. 그리고 불필요한 작업
        comments.append(line.strip())


In [140]:

print(comments)

['진짜 기여워잉', '망고처럼 꽃돌이가 너무 귀엽네옇', '귀여운 새침이', '꽃돌이 남자에요 여자에오', '꽃돌이는 여자에요', '아 너무 귀여워', '제가 키웠던 토끼가 자기 집 위에 올라가서 못 내려오던 이유가 있었군요', '이쁜것 아휴 꽃돌이 관절 소즁해', '이꽃돌씨와 우리 깜토 까미토끼 가만나면 어덯게 될까요 큰토끼님 답글 부탁드려용', '헉 감사해요 나중에 한번 될수있으면 꽃돌이보러갈께요', '깜토랑 만나면 좋은 친구가 될 수 있을 것 같아요', '꽃돌 왜 이렇게 귀여워서 난리양', '토끼한테 뽀뽀해주면 싫다고 앞발로 미는데 다리가 짧아서 집사는 같이 뽀뽀해주는 걸로 알고 좋아 죽음 토끼는 거절 했는데 계속해서 빡쳐 죽음', '꽃돌이도 이불을 씹는구나 톡톡톡하면서 망가지는 이불들이 남아 나질 않아', '덕분에 매 시즌 새로운 이불을 살 수 있다는 장점 이 있습니다', '꽃돌이 몸무게 몇 나가요', '꽃돌이가 살이많이쪘군요', '입니다', '꽃돌이는 얼굴이 동글동글', '앙 너무너문 너무너무 귀여워요', '졸귀', '귀여워요오', '침대위에서 무엇을 열심히 관측하는걸까요', '그러게말이에요', '인형인가요', '아이쿠 귀여운 꽂돌이', '귀 귀여워', '올라갈 땐 거침없이 올라가지만 내려올 땐 세상 쫄보도 그런 쫄보가 없는거 너무 공감이에요 저희 토끼도 소파 위에 올라가는걸 좋아하는데 매번 내려오지를 못해서 제가 밑에 누워있을 땐 항상 푹신한 제 뱃살로 뛰어내려오곤 해요', '본능적으로 가장 푹신한 곳을 찾아서 내려왔군요 똑똑한 뇨속', '꽃돌이는 귀도 도톰통통한게 너무귀여워요 이불탭은 맛있니', '앞다리는 짧지만 그 덕에 무척이나 귀엽고 앙증맞다는 사실 높은 곳 좋아하는 건 어쩔 수 없지만 내려오기 힘들다 싶으면 꼭 큰토끼님 도움을 받아야 한다 혼자서 막 내려오다 다치지 말고 손톱 하나 다쳐서 피 흘리는 모습조차도 보고싶지 않아', '짧아서 너무 귀여워 근데 또 높은 곳은 좋아하는 꽃돌이', '큰토끼님 에휴 나를 밟고 가시오', '귀여워 귀여워 

In [128]:
# COMMENT에서 명사, 형용사, 동사만 추출
com_noun = {}
com_verb = {}
com_adj = {}



for com in comments:
    tmp = okt.pos(com)

    # 품사 태그를 통해 명, 형, 동 구분
    for t in tmp:
        word, tag = t[0], t[1]
        if tag == 'Noun':
            if word in com_noun:
                com_noun[word] += 1
            else:
                com_noun[word] = 1

        elif tag == 'Verb':
            if word in com_verb:
                com_verb[word] += 1
            else:
                com_verb[word] = 1

        elif tag == 'Adjective':
            if word in com_adj:
                com_adj[word] += 1
            else:
                com_adj[word] = 1

com_noun = sorted(list(com_noun.items()), key= lambda x: (x[1], x[0]), reverse=True)
com_verb = sorted(list(com_verb.items()), key= lambda x: (x[1], x[0]), reverse=True)
com_adj = sorted(list(com_adj.items()), key= lambda x: (x[1], x[0]), reverse=True)

In [129]:
print(com_noun[:10])


[('돌이', 18), ('꽃', 17), ('토끼', 10), ('수', 4), ('제', 3), ('이불', 3), ('위', 3), ('앞다리', 3), ('땐', 3), ('꽃돌', 3)]


In [130]:
print(com_verb[:10])

[('큰', 4), ('해서', 2), ('않아', 2), ('만나면', 2), ('흘리는', 1), ('했는데', 1), ('해주면', 1), ('해주는', 1), ('해', 1), ('한다', 1)]


In [120]:
print(com_adj[:10])

[('같은', 175), ('없는', 120), ('입니다', 112), ('있는', 93), ('이런', 86), ('나쁜', 84), ('같은데', 83), ('없다', 79), ('좋은', 73), ('많은', 71)]


## DTM & TFIDF

In [131]:
# dtm class
import numpy as np

class DTM:
    def __init__(self):
        self.vocabulary = None


    def dtm_build(self, comments):
        comments_remake = []
        col_word = set()

        # 형태소 및 단어에서 단어 글자 크기가 1 이하이면 제외(sklearn과 동일하게 작동하도록 함)
        for com in comments:
            comments_remake.append([])
            tmp = com.split()

            for word in tmp:

                word_re = okt.pos(word)
                if len(word_re[0][0]) < 2:
                    continue
                col_word.add(word_re[0][0])
                comments_remake[-1].append(word_re[0][0])
        
        col_word = list(col_word)

        dtm = np.zeros(shape=(len(comments_remake), len(col_word)), dtype=np.int32) # row = comments, col = set of unique voca
        
        for i in range(len(comments_remake)):
            comment = comments_remake[i]
            for j in range(len(comment)):
                word = comment[j]
                if word in col_word:
                    idx = col_word.index(word)
                    dtm[i, idx] += 1


        self.vocabulary = {i:word for i, word in enumerate(col_word)}
        return dtm

            
dtm = DTM()
new_dtm = dtm.dtm_build(comments)
print(new_dtm)
# print(dtm.vocabulary)



[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [132]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

comments_remake = []
col_word = set()


for com in comments:
    tmp = com.split()
    strs = []
    for word in tmp:
        word_re = okt.pos(word)
        col_word.add(word_re[0][0])
        strs.append(word_re[0][0])
    comments_remake.append(' '.join(strs))
    
tfidfv = TfidfVectorizer().fit(comments_remake)
tfidf_matrix = tfidfv.transform(comments_remake).toarray()
voca_dict = tfidfv.vocabulary_


for i in range(tfidf_matrix.shape[0]):
    max_value = tfidf_matrix[i].argmax()
    m = tfidf_matrix[i].max()
    if  m > 0.7 and m < 1:
        print(list(voca_dict.keys())[list(voca_dict.values()).index(max_value)], m)

진짜 0.7422031318471273
남자 0.7422031318471273
귀여워 0.7856389514504228
관절 0.7071067811865475
나가요 0.7071067811865475
동글동글 0.7071067811865475
너무 0.8060093627548559
아이쿠 0.7422031318471273
귀여워 0.8420957014832894
기여웡 0.8518852253129331
짧은 0.7399133856298227
뭉툭 0.8518852253129331
